# Рубежный контроль №2

Тема: Методы обработки текстов.

Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на основе TfidfVectorizer.

In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [2]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

## Загрузка и первичный анализ данных

Используем данные из соревнования [Sentiment140.](http://help.sentiment140.com/for-students/)

In [3]:
# Загрузка данных sentiment140
sentiment140_df = pd.read_csv("trainingandtestdata/testdata.manual.2009.06.14.csv", delimiter=None, header=None)

The data is a CSV with emoticons removed. Data file format has 6 fields:
- 0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
- 1 - the id of the tweet (2087)
- 2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)
- 3 - the query (lyx). If there is no query, then this value is NO_QUERY.
- 4 - the user that tweeted (robotickilldozr)
- 5 - the text of the tweet (Lyx is cool)

In [4]:
sentiment140_df.head()

,0,1,2,3,4,5
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [5]:
sentiment140_df = pd.DataFrame(sentiment140_df,columns=[0,5])
sentiment140_df.columns = ['value','text']
# Удалить строку ’@id‘
sentiment140_df['text'] = sentiment140_df['text'].str.replace(r"\@[a-zA-Z0-9_]{1,}\s",'',regex=True)
# Удалить строку ‘#’
sentiment140_df['text'] = sentiment140_df['text'].str.replace('#','',regex=False)
sentiment140_df.head()

,value,text
0,4,I loooooooovvvvvveee my Kindle2. Not that the ...
1,4,Reading my kindle2... Love it... Lee childs i...
2,4,"Ok, first assesment of the kindle2 ...it fucki..."
3,4,You'll love your Kindle2. I've had mine for a ...
4,4,Fair enough. But i have the Kindle2 and I thi...


In [6]:
sentiment140_df.shape

(498, 2)

In [7]:
# Сформируем общий словарь для обучения моделей из обучающей и тестовой выборки
vocab_list = sentiment140_df['text'].tolist()
vocab_list[1:10]

['Reading my kindle2...  Love it... Lee childs is good read.',
 'Ok, first assesment of the kindle2 ...it fucking rocks!!!',
 "You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)",
 " Fair enough. But i have the Kindle2 and I think it's perfect  :)",
 "no. it is too big. I'm quite happy with the Kindle2.",
 'Fuck this economy. I hate aig and their non loan given asses.',
 'Jquery is my new best friend.',
 'Loves twitter',
 'how can you not love Obama? he makes jokes about himself.']

In [8]:
vocabVect = CountVectorizer()
vocabVect.fit(vocab_list)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 2174


In [9]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

my=1283
kindle2=1077
not=1339
that=1879
the=1880
dx=631
is=1029
cool=490
but=351


## Векторизация текста на основе модели "мешка слов"

Векторизация текста поддерживается библиотекой [scikit-learn.](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

### Использование класса [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) 

Подсчитывает количество слов словаря, входящих в данный текст.

In [10]:
test_features = vocabVect.transform(vocab_list)

In [11]:
test_features

<498x2174 sparse matrix of type '<class 'numpy.int64'>'
	with 6324 stored elements in Compressed Sparse Row format>

In [12]:
test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [13]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

2174

In [14]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2]

In [15]:
vocabVect.get_feature_names_out()[100:120]

array(['addictive', 'adidas', 'administration', 'admissions', 'adobe',
       'ads', 'advance', 'advice', 'aerospace', 'africa', 'after',
       'afternoon', 'again', 'agency', 'ago', 'ah', 'aha', 'ahead', 'ahh',
       'ahhh'], dtype=object)

### Использование класса [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) 

Вычисляет специфичность текста в корпусе текстов на основе метрики [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF).

In [16]:
tfidfv = TfidfVectorizer(ngram_range=(1,3))
tfidf_ngram_features = tfidfv.fit_transform(vocab_list)
tfidf_ngram_features

<498x12569 sparse matrix of type '<class 'numpy.float64'>'
	with 18019 stored elements in Compressed Sparse Row format>

In [17]:
tfidf_ngram_features.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
# Размер нулевой строки
len(tfidf_ngram_features.todense()[0].getA1())

12569

In [19]:
# Непустые значения нулевой строки
[i for i in tfidf_ngram_features.todense()[0].getA1() if i>0]

[0.0995053061454312,
 0.1436959778225222,
 0.16079126321438478,
 0.1381925251647641,
 0.16079126321438478,
 0.16079126321438478,
 0.1436959778225222,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.07817162915475566,
 0.16079126321438478,
 0.16079126321438478,
 0.12574165668347914,
 0.1436959778225222,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.11660059153729378,
 0.16079126321438478,
 0.16079126321438478,
 0.10279862873219407,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.16079126321438478,
 0.07418502268287902,
 0.12660069243065958,
 0.16079126321438478,
 0.09753119015737924,
 0.16079126321438478,
 0.16079126321438478,
 0.15079116232101897,
 0.16079126321438478,
 0.13369587692915638,
 0.09400185348767312,
 0.16079126321438478,
 0.16079126321438478,
 0.09989716414758186,
 0.15079116232101897,
 0.16079126321438478,
 0.16079126321438478,
 0.160791263214

## Решение задачи анализа тональности текста на основе модели "мешка слов"

С использованием кросс-валидации попробуем применить к корпусу текстов различные варианты векторизации и классификации.

In [20]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, sentiment140_df['text'], sentiment140_df['value'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

### Классификаторы
|  Классификатор №1  | Классификатор №2  |
|  ----  | ----  |
|LogisticRegression | Multinomial Naive Bayes - MNB

In [21]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
#classifiers_list = [LogisticRegression(C=3.0), LinearSVC(), KNeighborsClassifier()]
classifiers_list = [LogisticRegression(C=3.0), MultinomialNB(alpha=0.5)]
VectorizeAndClassify(vectorizers_list, classifiers_list)

Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '04fo': 2, '10': 3, '100': 4,
                            '1000': 5, '12': 6, '13t7nr': 7, '15mp': 8, '16': 9,
                            '16209': 10, '16szl1': 11, '17': 12, '1796': 13,
                            '1988': 14, '19epah': 15, '19j2d': 16, '1aikhf': 17,
                            '1st': 18, '1zlff': 19, '200': 20, '2009': 21,
                            '2010': 22, '2012': 23, '22': 24, '24': 25,
                            '299': 26, '2boqu': 27, '2cuig': 28, '2day': 29, ...})
Модель для классификации - LogisticRegression(C=3.0)
Accuracy = 0.6506024096385542
Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '04fo': 2, '10': 3, '100': 4,
                            '1000': 5, '12': 6, '13t7nr': 7, '15mp': 8, '16': 9,
                            '16209': 10, '16szl1': 11, '17': 12, '1796': 13,
                            '1988': 14, '19epah': 15, '19j2d': 16, '1aikhf': 17,
                         

### Разделим выборку на обучающую и тестовую и проверим решение для лучшей модели

In [22]:
X_train, X_test, y_train, y_test = train_test_split(sentiment140_df['text'], sentiment140_df['value'], test_size=0.5, random_state=1)

In [23]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [24]:
sentiment(TfidfVectorizer(), LogisticRegression(C=5.0))

Метка 	 Accuracy
0 	 0.8625
2 	 0.6911764705882353
4 	 0.5346534653465347


In [25]:
sentiment(CountVectorizer(), LogisticRegression(C=5.0))

Метка 	 Accuracy
0 	 0.75
2 	 0.6617647058823529
4 	 0.5643564356435643


In [26]:
sentiment(TfidfVectorizer(), MultinomialNB(alpha=0.5))

Метка 	 Accuracy
0 	 0.8875
2 	 0.45588235294117646
4 	 0.5445544554455446


In [27]:
sentiment(CountVectorizer(), MultinomialNB(alpha=0.5))

Метка 	 Accuracy
0 	 0.8
2 	 0.5294117647058824
4 	 0.594059405940594


In [28]:
sentiment(TfidfVectorizer(), MultinomialNB(alpha=0.85))

Метка 	 Accuracy
0 	 0.9
2 	 0.38235294117647056
4 	 0.44554455445544555


In [29]:
sentiment(CountVectorizer(), MultinomialNB(alpha=0.85))

Метка 	 Accuracy
0 	 0.85
2 	 0.5294117647058824
4 	 0.5247524752475248
